In [ ]:
!pip install statsmodels

In [ ]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import glob
import os
import fnmatch
import pandas as pd
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib as mplt
import scipy.stats as st
import statsmodels.stats.api as sms
import colorsys
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter)
import math

# CIFAR100, scaling crossvalidation 

ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180423_132821_mini_imagenet_metric_multiplier_trainable_encoder_classifier_link_feat_extract_pretrain_metric_multiplier_init_number_of_steps_cbn_per_block_cbn_num_layers_repeat_metric_scaling_crossvalidation_CIFAR100'
x_axis_vals = [0.5, 1, 5, 10, 20]
number_of_steps = 10000
feat_extract_pretrain='multitask'  # None,'multitask'
encoder_classifier_link='cbn'
experiement_patterns = [
    
    '*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d;*metric_multiplier_init=0.5' 
    %(feat_extract_pretrain,encoder_classifier_link,number_of_steps),
    '*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d;*metric_multiplier_init=1'
    %(feat_extract_pretrain,encoder_classifier_link,number_of_steps),
    '*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d;*metric_multiplier_init=5'
    %(feat_extract_pretrain,encoder_classifier_link,number_of_steps),
    '*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d;*metric_multiplier_init=10'
    %(feat_extract_pretrain,encoder_classifier_link,number_of_steps),
    '*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d;*metric_multiplier_init=20'
    %(feat_extract_pretrain,encoder_classifier_link,number_of_steps),
]

# MINIIMAGENET, 5-shot
# This is the first result
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180422_013228_mini_imagenet_metric_multiplier_trainable_encoder_classifier_link_feat_extract_pretrain_polynomial_metric_order_metric_multiplier_init_cbn_per_block_cbn_num_layers_repeat_scaling_crossvalidation_min'
# Scaling paramter crossvalidation, polynomial head, TBN, WITH multitask
x_axis_vals = [0.5, 1, 5, 10, 20]
experiement_patterns = [
    '*cbn_num_layers=3*pretrain=multitask*encoder_classifier_link=cbn*cbn_per_block=False*multiplier_init=0.5',
    '*cbn_num_layers=3*pretrain=multitask*encoder_classifier_link=cbn*cbn_per_block=False*multiplier_init=1',
    '*cbn_num_layers=3*pretrain=multitask*encoder_classifier_link=cbn*cbn_per_block=False*multiplier_init=5',
    '*cbn_num_layers=3*pretrain=multitask*encoder_classifier_link=cbn*cbn_per_block=False*multiplier_init=10',
    '*cbn_num_layers=3*pretrain=multitask*encoder_classifier_link=cbn*cbn_per_block=False*multiplier_init=20',
]
# This is a bit better result with the last maxpool layer before avg pool removed
ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180508_195508_mini_imagenet_num_max_pools_encoder_classifier_link_feat_extract_pretrain_metric_multiplier_init_cbn_num_layers_repeat_scale_maxpool3'
x_axis_vals = [0.5, 1, 5, 7.5, 10, 20]
experiement_patterns = [
    '*cbn_num_layers=3*pretrain=multitask*encoder_classifier_link=cbn*multiplier_init=0.5',
    '*cbn_num_layers=3*pretrain=multitask*encoder_classifier_link=cbn*multiplier_init=1',
    '*cbn_num_layers=3*pretrain=multitask*encoder_classifier_link=cbn*multiplier_init=5',
    '*cbn_num_layers=3*pretrain=multitask*encoder_classifier_link=cbn*multiplier_init=7.5',
    '*cbn_num_layers=3*pretrain=multitask*encoder_classifier_link=cbn*multiplier_init=10',
    '*cbn_num_layers=3*pretrain=multitask*encoder_classifier_link=cbn*multiplier_init=20',
]
# # Scaling paramter crossvalidation, polynomial head, TBN, WITHOUT multitask
# experiement_patterns = [
#     '*cbn_num_layers=3*pretrain=None*encoder_classifier_link=cbn*cbn_per_block=False*multiplier_init=0.5',
#     '*cbn_num_layers=3*pretrain=None*encoder_classifier_link=cbn*cbn_per_block=False*multiplier_init=1',
#     '*cbn_num_layers=3*pretrain=None*encoder_classifier_link=cbn*cbn_per_block=False*multiplier_init=5',
#     '*cbn_num_layers=3*pretrain=None*encoder_classifier_link=cbn*cbn_per_block=False*multiplier_init=10',
#     '*cbn_num_layers=3*pretrain=None*encoder_classifier_link=cbn*cbn_per_block=False*multiplier_init=20',
# ]
# # The scaling parameter crossvalidation for polynomial head, WITHOUT multitask, (two times more steps!)
# experiement_patterns = [
#     '*pretrain=None*encoder_classifier_link=polynomial*number_of_steps=60000*multiplier_init=0.5',
#     '*pretrain=None*encoder_classifier_link=polynomial*number_of_steps=60000*multiplier_init=1',
#     '*pretrain=None*encoder_classifier_link=polynomial*number_of_steps=60000*multiplier_init=5',
#     '*pretrain=None*encoder_classifier_link=polynomial*number_of_steps=60000*multiplier_init=10',
#     '*pretrain=None*encoder_classifier_link=polynomial*number_of_steps=60000*multiplier_init=20',
# ]
# # The scaling parameter crossvalidation for polynomial head, WITH multitask, 30000 steps
# experiement_patterns = [
#     '*pretrain=multitask*encoder_classifier_link=polynomial*multiplier_init=0.5',
#     '*pretrain=multitask*encoder_classifier_link=polynomial*multiplier_init=1',
#     '*pretrain=multitask*encoder_classifier_link=polynomial*multiplier_init=5',
#     '*pretrain=multitask*encoder_classifier_link=polynomial*multiplier_init=10',
#     '*pretrain=multitask*encoder_classifier_link=polynomial*multiplier_init=20',
# ]



# ### 1-shot results
# # MINIIMAGENET
# ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180503_102004_mini_imagenet_num_tasks_per_batch_encoder_classifier_link_feat_extract_pretrain_train_batch_size_metric_multiplier_init_number_of_steps_cbn_num_layers_repeat_scale_1shot_first_run'
# x_axis_vals = [0.2, 0.5, 1, 2.5, 5]
# cbn_num_layers=3
# number_of_steps=40000
# encoder_classifier_link='polynomial'  # 'polynomial', 'cbn'
# feat_extract_pretrain='None'  # None 'multitask'
# experiement_patterns = [
#     '*cbn_num_layers=%d*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d*metric_multiplier_init=0.2' 
#     %(cbn_num_layers, feat_extract_pretrain, encoder_classifier_link, number_of_steps),
#     '*cbn_num_layers=%d*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d*metric_multiplier_init=0.5' 
#     %(cbn_num_layers, feat_extract_pretrain, encoder_classifier_link, number_of_steps),
#     '*cbn_num_layers=%d*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d*metric_multiplier_init=1' 
#     %(cbn_num_layers, feat_extract_pretrain, encoder_classifier_link, number_of_steps),
#     '*cbn_num_layers=%d*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d*metric_multiplier_init=2.5' 
#     %(cbn_num_layers, feat_extract_pretrain, encoder_classifier_link, number_of_steps),
#     '*cbn_num_layers=%d*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d*metric_multiplier_init=5' 
#     %(cbn_num_layers, feat_extract_pretrain, encoder_classifier_link, number_of_steps),
# ]


# ### 1-shot results
# # CIFAR-100
# ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180505_022014_mini_imagenet_num_tasks_per_batch_encoder_classifier_link_feat_extract_pretrain_train_batch_size_metric_multiplier_init_number_of_steps_cbn_num_layers_repeat_scale_crossvalidation_CIFAR100_1shot'
# x_axis_vals = [0.5, 1, 1.5, 2, 2.5]
# cbn_num_layers=3
# number_of_steps=10000
# encoder_classifier_link='polynomial'  # 'polynomial', 'cbn'
# feat_extract_pretrain='None'  # None 'multitask'
# experiement_patterns = [
#     '*cbn_num_layers=%d*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d*metric_multiplier_init=0.5' 
#     %(cbn_num_layers, feat_extract_pretrain, encoder_classifier_link, number_of_steps),
#     '*cbn_num_layers=%d*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d*metric_multiplier_init=1' 
#     %(cbn_num_layers, feat_extract_pretrain, encoder_classifier_link, number_of_steps),
#     '*cbn_num_layers=%d*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d*metric_multiplier_init=1.5' 
#     %(cbn_num_layers, feat_extract_pretrain, encoder_classifier_link, number_of_steps),
#     '*cbn_num_layers=%d*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d*metric_multiplier_init=2' 
#     %(cbn_num_layers, feat_extract_pretrain, encoder_classifier_link, number_of_steps),
#     '*cbn_num_layers=%d*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d*metric_multiplier_init=2.5' 
#     %(cbn_num_layers, feat_extract_pretrain, encoder_classifier_link, number_of_steps),
# ]


# ### 10-shot results
# # MINIIMAGENET
# ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180506_020918_mini_imagenet_encoder_classifier_link_feat_extract_pretrain_metric_multiplier_init_init_learning_rate_number_of_steps_cbn_num_layers_repeat_scale_10shot'
# x_axis_vals = [0.5, 1, 5, 10, 15, 20]
# cbn_num_layers=3
# number_of_steps=60000
# encoder_classifier_link='polynomial'  # 'polynomial', 'cbn'
# feat_extract_pretrain='None'  # None 'multitask'
# experiement_patterns = [
#     '*cbn_num_layers=%d*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d*metric_multiplier_init=0.5' 
#     %(cbn_num_layers, feat_extract_pretrain, encoder_classifier_link, number_of_steps),
#     '*cbn_num_layers=%d*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d*metric_multiplier_init=1' 
#     %(cbn_num_layers, feat_extract_pretrain, encoder_classifier_link, number_of_steps),
#     '*cbn_num_layers=%d*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d*metric_multiplier_init=5' 
#     %(cbn_num_layers, feat_extract_pretrain, encoder_classifier_link, number_of_steps),
#     '*cbn_num_layers=%d*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d*metric_multiplier_init=10' 
#     %(cbn_num_layers, feat_extract_pretrain, encoder_classifier_link, number_of_steps),
#     '*cbn_num_layers=%d*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d*metric_multiplier_init=15' 
#     %(cbn_num_layers, feat_extract_pretrain, encoder_classifier_link, number_of_steps),
#     '*cbn_num_layers=%d*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d*metric_multiplier_init=20' 
#     %(cbn_num_layers, feat_extract_pretrain, encoder_classifier_link, number_of_steps),
# ]

# ### 10-shot results
# # CIFAR100
# ROOT_DIR='/mnt/home/boris/experiments_task_encoder/180507_015250_mini_imagenet_num_tasks_per_batch_encoder_classifier_link_feat_extract_pretrain_metric_multiplier_init_init_learning_rate_number_of_steps_cbn_num_layers_repeat_scale_crossval_CIFAR100_10shot'
# x_axis_vals = [0.5, 1, 5, 10, 15, 20]
# cbn_num_layers=3
# number_of_steps=20000
# encoder_classifier_link='polynomial'  # 'polynomial', 'cbn'
# feat_extract_pretrain='None'  # None 'multitask'
# experiement_patterns = [
#     '*cbn_num_layers=%d*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d*metric_multiplier_init=0.5' 
#     %(cbn_num_layers, feat_extract_pretrain, encoder_classifier_link, number_of_steps),
#     '*cbn_num_layers=%d*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d*metric_multiplier_init=1' 
#     %(cbn_num_layers, feat_extract_pretrain, encoder_classifier_link, number_of_steps),
#     '*cbn_num_layers=%d*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d*metric_multiplier_init=5' 
#     %(cbn_num_layers, feat_extract_pretrain, encoder_classifier_link, number_of_steps),
#     '*cbn_num_layers=%d*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d*metric_multiplier_init=10' 
#     %(cbn_num_layers, feat_extract_pretrain, encoder_classifier_link, number_of_steps),
#     '*cbn_num_layers=%d*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d*metric_multiplier_init=15' 
#     %(cbn_num_layers, feat_extract_pretrain, encoder_classifier_link, number_of_steps),
#     '*cbn_num_layers=%d*feat_extract_pretrain=%s;encoder_classifier_link=%s;number_of_steps=%d*metric_multiplier_init=20' 
#     %(cbn_num_layers, feat_extract_pretrain, encoder_classifier_link, number_of_steps),
# ]







In [ ]:
import tensorflow as tf; print(tf.__version__)

# Get directories for the  experiements 

In [ ]:
def get_experiment_folder_list(experiement_dir, experiment_pattern):
    list_experiemnts_pattern = []
    list_experiments_all = os.listdir(experiement_dir)
    for folder in list_experiments_all:
        names = os.path.join(ROOT_DIR,folder)
#         print(names)
        names_filt_alone = fnmatch.filter([names], experiment_pattern)
#         print(names_filt_alone)
        list_experiemnts_pattern.extend(names_filt_alone)
    return list_experiemnts_pattern


def interpolate_data(experiment_data, index_max, step):
    index_max = int(round(index_max/step)*step)
    index_new = np.arange(0, index_max+step, step)
    experiment_data_interpolated = {}
    for file, df in experiment_data.items():
        interpolator = interpolate.interp1d(df.index, df.value, bounds_error=False, fill_value=np.nan)
        value_new = interpolator(index_new)
        experiment_data_interpolated[file] = value_new
    
    return pd.DataFrame(index=index_new, data=experiment_data_interpolated)
    

def read_experiment_data(experiement_dir, experiment_pattern):
    experiemnt_folder_list = get_experiment_folder_list(experiement_dir, experiment_pattern)
    experiment_data_tst = {}
    experiment_data_val = {}
    index_max_tst = 0
    index_max_val = 0
    
    i=0    
    for experiement in experiemnt_folder_list:
        print(experiement)
        print(glob.glob1(experiement+'/eval/', 'events.out.tfevents.*'))
        df_tst=[]
        df_val=[]
        for file in glob.glob1(experiement+'/eval/', 'events.out.tfevents.*'):
            ea = EventAccumulator(experiement+'/eval/'+file)
            ea.Reload()
            tags = ea.Tags()['scalars']
#             print(tags)
            if not 'accuracy_target_tst' in tags:
                continue
            data = ea.Scalars('accuracy_target_tst')
            if len(data) > 0:
                df_tst.append(pd.DataFrame(data).set_index('step').drop('wall_time', axis=1))
            data = ea.Scalars('accuracy_target_val')
            if len(data) > 0:
                df_val.append(pd.DataFrame(data).set_index('step').drop('wall_time', axis=1))
        
        if len(df_tst) > 0:
            df_tst = pd.concat(df_tst)

        if len(df_val) > 0:
            df_val = pd.concat(df_val)
        
        if len(df_tst) > 1:
            experiment_data_tst[experiement+file] = df_tst
            index_max_tst = max(index_max_tst, df_tst.index.max())
        
        if len(df_tst) > 1:
            experiment_data_val[experiement+file] = df_val
            index_max_val = max(index_max_val, df_val.index.max())
        
    return interpolate_data(experiment_data_tst, index_max_tst, step=250), interpolate_data(experiment_data_val, index_max_val, step=250)


In [ ]:
list_experiemnts_target_transfer = get_experiment_folder_list(ROOT_DIR, experiement_patterns[0])
list_experiemnts_target_transfer

In [ ]:
plot_data = {}
for experiment_pattern in experiement_patterns:
    print('Loading experiment: %s' %experiment_pattern)
    acc_tst, acc_val = read_experiment_data(ROOT_DIR, experiment_pattern)
    
    plot_data[(experiment_pattern, 'acc_tst')] = acc_tst
    plot_data[(experiment_pattern, 'acc_val')] = acc_val

In [ ]:
# This is to detect runs that did not converge
VALID_RUN_THRESHOLD=0.0

for experiment_pattern in experiement_patterns:
    print('Loading experiment: %s' %experiment_pattern)
    
    acc_tst = plot_data[(experiment_pattern, 'acc_tst')]
    acc_val = plot_data[(experiment_pattern, 'acc_val')]
    
#     acc_tst = acc_tst[acc_tst.index<=25000]
#     acc_val = acc_val[acc_val.index<=25000]

#     columns = list(acc_tst.columns[0:2]) + list(acc_tst.columns[2:-1])
#     acc_tst = acc_tst[columns]
#     acc_val = acc_val[columns]
    
    valid_cols = acc_val.max().index[acc_val.max() > VALID_RUN_THRESHOLD]
    acc_val = acc_val[valid_cols]
    acc_tst = acc_tst[valid_cols]
    
    acc_tst_ci = sms.DescrStatsW(acc_tst.T).tconfint_mean()
    acc_val_ci = sms.DescrStatsW(acc_val.T).tconfint_mean()
    acc_val_mean = acc_val.mean(axis=1)
    acc_tst_mean = acc_tst.mean(axis=1)
    
    plot_data[(experiment_pattern, 'acc_tst_mean')] = acc_tst_mean
    plot_data[(experiment_pattern, 'acc_val_mean')] = acc_val_mean
    plot_data[(experiment_pattern, 'acc_tst_ci')] = acc_tst_ci
    plot_data[(experiment_pattern, 'acc_val_ci')] = acc_val_ci
    plot_data[(experiment_pattern, 'acc_tst')] = acc_tst
    plot_data[(experiment_pattern, 'acc_val')] = acc_val

In [ ]:
def argmax_to_max(arr, argmax, axis):
    """argmax_to_max(arr, arr.argmax(axis), axis) == arr.max(axis)"""
    new_shape = list(arr.shape)
    del new_shape[axis]

    grid = np.ogrid[tuple(map(slice, new_shape))]
    grid.insert(axis, argmax)

    return arr[tuple(grid)]


def get_max_over_val_bootstrapped(acc_tst, acc_val, bootstrap_ratio, n_mc_trials, replace):
    acc_tst = acc_tst.values
    acc_val = acc_val.values
    assert acc_val.shape[1] == acc_tst.shape[1]
    output = np.zeros(shape=(acc_tst.shape[0], n_mc_trials))
    for i in range(n_mc_trials):
        col_dxs = np.random.choice(range(acc_val.shape[1]), size=round(bootstrap_ratio*acc_val.shape[1]), 
                                  replace=replace)
        acc_val_boot = acc_val[:,col_dxs]
        acc_tst_boot = acc_tst[:,col_dxs]
                
        val_max_idx = np.argmax(acc_val_boot, axis=1)
        output[:, i] = argmax_to_max(acc_tst_boot, val_max_idx, axis=1)

    return output
        
    
        

In [ ]:
def get_early_stop_over_val(acc_tst, acc_val):
    acc_tst = np.nan_to_num(acc_tst.values)
    acc_val = np.nan_to_num(acc_val.values)
    assert acc_val.shape[1] == acc_tst.shape[1]
        
    val_max_idx = np.argmax(acc_val, axis=0)
    acc_tst_max = argmax_to_max(acc_tst, val_max_idx, axis=0)
    
    return acc_tst_max

print("Experiment root folder: %s" %ROOT_DIR)
for i, experiment_pattern in enumerate(experiement_patterns):
    acc_tst, acc_val = plot_data[(experiment_pattern, 'acc_tst')], plot_data[(experiment_pattern, 'acc_val')]
    acc_tst_validated = get_early_stop_over_val(acc_tst, acc_val)
    
    acc_tst_validated_ci = sms.DescrStatsW(acc_tst_validated.T).tconfint_mean()
    acc_tst_validated_mean = acc_tst_validated.mean()
    
    print('Experiment name: %s' %(experiment_pattern))
    print('mean accuracy: %.1f +/- %.1f' %(100.0*acc_tst_validated_mean, 
            100.0*(acc_tst_validated_ci[1]-acc_tst_validated_mean)))


In [ ]:
print("Experiment root folder: %s" %ROOT_DIR)
for i, experiment_pattern in enumerate(experiement_patterns):
    acc_tst, acc_val = plot_data[(experiment_pattern, 'acc_tst')], plot_data[(experiment_pattern, 'acc_val')]
    acc_tst_validated = get_early_stop_over_val(acc_val, acc_val)
    
    acc_tst_validated_ci = sms.DescrStatsW(acc_tst_validated.T).tconfint_mean()
    acc_tst_validated_mean = acc_tst_validated.mean()
    
    print('Experiment name: %s' %(experiment_pattern))
    print('mean accuracy: %.1f +/- %.1f' %(100.0*acc_tst_validated_mean, 
            100.0*(acc_tst_validated_ci[1]-acc_tst_validated_mean)))

In [ ]:
mplt.rcParams.update({'font.size': 22})

majorLocator = MultipleLocator(1.0)
majorFormatter = FormatStrFormatter('%.1f')
colors=[colorsys.hsv_to_rgb(x*1.0/2.0, 1.0, 1.0) for x in range(2)]

fig = plt.figure(figsize=(20,5))
ax1 = fig.add_subplot(1,1,1)
plt.grid('on')

acc_val_vs_degree_mean = np.zeros(len(x_axis_vals))
acc_val_vs_degree_ci = np.zeros((len(x_axis_vals), 2))
acc_tst_vs_degree_mean = np.zeros(len(x_axis_vals))
acc_tst_vs_degree_ci = np.zeros((len(x_axis_vals), 2))
for i, experiment_pattern in enumerate(experiement_patterns):
    acc_tst, acc_val = plot_data[(experiment_pattern, 'acc_tst')], plot_data[(experiment_pattern, 'acc_val')]
    
    # This is based on early stop over each run
    acc_tst_validated = get_early_stop_over_val(acc_tst, acc_val)
    acc_val_validated = get_early_stop_over_val(acc_val, acc_val)
    
#     # This is based on early stop over the mean
#     acc_val_mean = acc_val.mean(axis=1)
#     stop_idx = acc_val_mean.argmax()
#     acc_val_validated = acc_val.loc[stop_idx]
#     acc_tst_validated = acc_tst.loc[stop_idx]
    
    acc_val_vs_degree_ci[i] = sms.DescrStatsW(acc_val_validated.T).tconfint_mean()
    acc_val_vs_degree_mean[i] = acc_val_validated.mean()
    
    acc_tst_vs_degree_ci[i] = sms.DescrStatsW(acc_tst_validated.T).tconfint_mean()
    acc_tst_vs_degree_mean[i] = acc_tst_validated.mean()
    
    
ax1.plot(x_axis_vals, 100*acc_val_vs_degree_mean, label='validation', c=colors[0])
ax1.fill_between(x_axis_vals, 100.0*acc_val_vs_degree_ci[:,0], 100.0*acc_val_vs_degree_ci[:,1], 
                alpha=0.25, edgecolor=colors[0], facecolor=colors[0])

ax1.plot(x_axis_vals, 100*acc_tst_vs_degree_mean, label='test', c=colors[1])
ax1.fill_between(x_axis_vals, 100.0*acc_tst_vs_degree_ci[:,0], 100.0*acc_tst_vs_degree_ci[:,1], 
                alpha=0.25, edgecolor=colors[1], facecolor=colors[1])

ylim_min = 100*min(acc_val_vs_degree_mean.min(), acc_tst_vs_degree_mean.min()) - 0.5
ylim_max = 100*max(acc_val_vs_degree_ci.max(), acc_tst_vs_degree_ci.max()) + 0.5

legend = ax1.legend(shadow=True, loc=4)
ax1.grid(which='minor', alpha=0.5)                                                
ax1.grid(which='major', alpha=1.0) 
ax1.set_ylim([ylim_min,ylim_max])
ax1.set_xlim([0.0,max(x_axis_vals)+0.5])

ax1.set_xlabel(r'Scale, $\alpha$')
ax1.set_xticks(range(1, math.floor(max(x_axis_vals))+1))
ax1.set_ylabel('Accuracy, %')
ax1.yaxis.set_major_locator(majorLocator)
ax1.yaxis.set_major_formatter(majorFormatter)


In [ ]:
acc_val_vs_degree_mean*100

In [ ]:
acc_tst_vs_degree_mean*100